In [ ]:
#| default_exp tui


# TUI - Terminal User Interface

Textual-based terminal user interface components for audio transcription.


In [ ]:
#| export
from enum import Enum, auto
import asyncio
import pyperclip
import os

from textual import on
from textual.app import App, ComposeResult
from textual.reactive import reactive
from textual.containers import Container, Grid, Center, Horizontal, Vertical, HorizontalGroup
from textual.screen import ModalScreen
from textual.widgets import Header, Footer, Static, Button, Log, Select, Rule, MarkdownViewer, Input, LoadingIndicator

from tui_writer.live import LiveTranscriber
from tui_writer.ai import TranscriptEditor
from lisette import Chat

In [ ]:
#| export

TEXTUAL_CSS = """
    Screen {
        align: center top;
    }
    Header {
        background: $background;
    }
    .recording {
        background: $error;
    }

    #transcript-display {
        hatch: horizontal $boost 80%;
        background: $boost;
        margin: 1 3;
        padding: 2 3;
    }
    
    HelpModal {
        align: center middle;
    }
    #help {
        border: thick $background 80%;
        background: $surface;
        width: 100;
        padding: 0 2;
        align: center top;
        height: 90%;
        max-height: 70;
    }

    
    SettingsModal {
        align: center middle;
    }
    #settings-modal {
        border: thick $background 80%;
        background: $surface;
        width: 100;
        padding: 0 2;
        align: center top;
        height: 20;
    }
    
    AISettingsModal {
        align: center middle;
    }
    #ai-settings-modal {
        border: thick $background 80%;
        background: $surface;
        width: 120;
        padding: 0 2;
        align: center top;
        height: 40;
    }
    
    #modal-header {
        content-align: center middle;
        margin: 1 0;
    }
    Rule {
        margin: 0;
    }
    #settings-grid {
        layout: grid;
        grid-size: 3 3;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
    }
    #ai-settings-grid {
        layout: grid;
        grid-size: 3;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
    }

    .select-text {
        height: 100%;
        content-align: center middle;
    }
    Select {
        column-span: 2;
    }
    Input {
        column-span: 2;
    }
    #settings-actions {
        column-span: 3;
    }
    #ai-settings-actions {
        width: auto;
    }
"""

In [ ]:
#| export

class SettingsModal(ModalScreen):
    # Available Whisper model options (label, value)
    WHISPER_MODELS = [
        ("Tiny      ⚡  Ultra fast   - Low accuracy", "tiny"),
        ("Base      ⚡  Fast         - Decent accuracy", "base"),
        ("Small     ⚖   Balanced    - Good accuracy", "small"),
        ("Medium    🐢  Slow        - High accuracy",  "medium"),
        ("Large     🐢  Very Slow   - Best accuracy", "large-v3"),
        ("Tiny.en   English-only    - faster", "tiny.en"),
        ("Base.en   English-only    - faster", "base.en"),
        ("Small.en  English-only    - faster", "small.en"),
        ("Medium.en English-only    - faster", "medium.en"),
        ("Large.en  English-only    - faster", "large-v3.en")
    ]
    WHISPER_LANGUAGES = [
        ("English", "en"),
        ("Norwegian", "no"),
        ("Swedish", "sv"),
        ("Danish", "da"),
        ("Finnish", "fi"),
        ("German", "de"),
        ("Dutch", "nl"),
        ("French", "fr"),
        ("Spanish", "es"),
        ("Portuguese", "pt"),
        ("Italian", "it"),
        ("Polish", "pl"),
        ("Czech", "cs"),
        ("Slovak", "sk"),
        ("Hungarian", "hu"),
        ("Greek", "el"),
        ("Turkish", "tr"),
        ("Russian", "ru"),
        ("Ukrainian", "uk"),
        ("Arabic", "ar"),
        ("Hebrew", "he"),
        ("Hindi", "hi"),
        ("Bengali", "bn"),
        ("Urdu", "ur"),
        ("Persian (Farsi)", "fa"),
        ("Thai", "th"),
        ("Vietnamese", "vi"),
        ("Indonesian", "id"),
        ("Malay", "ms"),
        ("Filipino (Tagalog)", "tl"),
        ("Chinese (Mandarin)", "zh"),
        ("Chinese (Cantonese)", "yue"),
        ("Japanese", "ja"),
        ("Korean", "ko"),
        ("Swahili", "sw"),
        ("Afrikaans", "af"),
        ("Romanian", "ro"),
        ("Bulgarian", "bg"),
        ("Serbian", "sr"),
        ("Croatian", "hr"),
        ("Slovenian", "sl"),
        ("Estonian", "et"),
        ("Latvian", "lv"),
        ("Lithuanian", "lt"),
    ]

    def __init__(self, model, language):
        self.model = model
        self.language = language
        super().__init__()

    def compose(self) -> ComposeResult:
        with Container(id="settings-modal"):
            yield Static("SETTINGS", id="modal-header")
            yield Rule(line_style="heavy")
            with Grid(id="settings-grid"):
                yield Static("Transcribing model:", classes="select-text")
                yield Select(
                        options=self.WHISPER_MODELS,
                        allow_blank=False,
                        value=self.model,
                        id="whisper-model"
                )
                yield Static("Language:", classes="select-text")
                yield Select(
                        options=self.WHISPER_LANGUAGES,
                        allow_blank=False,
                        value=self.language,
                        id="whisper-language"
                )
                yield Center(
                    Button.success("Close (esc)", id="save"),
                    id="settings-actions",
                )


    # Whisper selectors
    @on(Select.Changed, "#whisper-model")
    def on_model_changed(self, event: Select.Changed) -> None:
        self.model = str(event.value)

    @on(Select.Changed, "#whisper-language")
    def on_language_changed(self, event: Select.Changed) -> None:
        self.language = str(event.value)

    def key_escape(self) -> None:
        self.dismiss([self.model, self.language])

    def on_button_pressed(self) -> None:
        self.dismiss([self.model, self.language])


In [ ]:
#| export

class AISettingsModal(ModalScreen):
    """Modal for managing AI model settings across multiple providers."""

    PROVIDER_OPTIONS = [
        ("OpenAI", "openai"),
        ("Anthropic", "anthropic"),
        ("Google Gemini", "gemini"),
    ]
    
    # Provider-specific model options
    PROVIDER_MODELS = {
        "openai": [
            ("GPT-4o", "gpt-4o"),
            ("GPT-4o Mini", "gpt-4o-mini"),
            ("GPT-4 Turbo", "gpt-4-turbo"),
            ("GPT-4", "gpt-4"),
            ("GPT-3.5 Turbo", "gpt-3.5-turbo"),
        ],
        "anthropic": [
            ("Claude 3.5 Sonnet", "claude-3-5-sonnet-20241022"),
            ("Claude 3 Opus", "claude-3-opus-20240229"),
            ("Claude 3 Sonnet", "claude-3-sonnet-20240229"),
            ("Claude 3 Haiku", "claude-3-haiku-20240307"),
        ],
        "gemini": [
            ("Gemini 1.5 Pro", "gemini-1.5-pro"),
            ("Gemini 1.5 Flash", "gemini-1.5-flash"),
            ("Gemini 1.0 Pro", "gemini-1.0-pro"),
        ],
    }

    def __init__(self, current_provider: str = "openai", current_model: str = "gpt-4o-mini"):
        super().__init__()
        self.provider = current_provider
        self.current_model = current_model
        
        # Get model options for current provider
        self.model_options = self.PROVIDER_MODELS.get(self.provider, [])

    def compose(self) -> ComposeResult:
        with Container(id="ai-settings-modal"):
            yield Static("AI MODEL SETTINGS", id="modal-header")
            yield Rule(line_style="heavy")
            
            with Grid(id="ai-settings-grid"):
                # Provider selection
                yield Static("AI Provider:", classes="select-text")
                yield Select(
                    options=self.PROVIDER_OPTIONS,
                    allow_blank=False,
                    value=self.provider,
                    id="ai-provider"
                )
                
                # Model selection
                yield Static("Model:", classes="select-text")
                yield Select(
                    options=self.model_options,
                    allow_blank=False,
                    value=self.current_model if self.model_options else None,
                    id="ai-model"
                )
                
                # API Key input
                yield Static("API Key:", classes="select-text", id="api-key-text")
                yield Input(
                    placeholder=f"Enter {self.provider.title()} API key...",
                    password=True,
                    id="api-key-input"
                )
            
            with Center():
                yield HorizontalGroup(
                    Button("Apply", id="apply-ai-settings", variant="success"),
                    Button("Cancel", id="cancel-ai-settings", variant="default"),
                    id="ai-settings-actions"
                )

    def on_mount(self) -> None:
        self.model_select = self.query_one("#ai-model", Select)
        self.api_key_input = self.query_one("#api-key-input", Input)
        self.api_key_text = self.query_one("#api-key-text", Static)

    @on(Select.Changed, "#ai-provider")
    def on_provider_changed(self, event: Select.Changed) -> None:
        if self.provider == str(event.value):
            return

        self.provider = str(event.value)
        
        # Update model options for the selected provider
        self.model_options = self.PROVIDER_MODELS.get(self.provider, [])
        self.model_select.set_options(self.model_options)
        
        # Set a default model if available
        if self.model_options:
            self.current_model = self.model_options[0][1]
        
        # Update placeholder text
        self.api_key_input.placeholder = f"Enter {self.provider.title()} API key..."

    @on(Select.Changed, "#ai-model")
    def on_model_changed(self, event: Select.Changed) -> None:
        if self.current_model == str(event.value):
            return
        self.current_model = str(event.value)

    @on(Button.Pressed, "#apply-ai-settings")
    def on_apply_settings(self) -> None:
        api_key = self.api_key_input.value.strip()
        
        if not api_key:
            self.notify(f"API key required for {self.provider.title()}", severity="warning", title="Missing API Key")
            return
        
        # Save API key to environment variable
        env_var_map = {
            "openai": "OPENAI_API_KEY",
            "anthropic": "ANTHROPIC_API_KEY",
            "gemini": "GEMINI_API_KEY"
        }
        
        env_var = env_var_map.get(self.provider)
        if env_var:
            os.environ[env_var] = api_key
            self.notify(
                f"Switched to {self.provider.title()}: {self.current_model}", 
                title="AI Provider Updated"
            )
            self.dismiss((self.provider, self.current_model))
        else:
            self.notify("Unknown provider", severity="error", title="Setup Failed")

    @on(Button.Pressed, "#cancel-ai-settings")
    def on_cancel_settings(self) -> None:
        self.dismiss(None)

    def key_escape(self) -> None:
        self.dismiss(None)


In [ ]:
#| export
HELP_MARKDOWN = """\
# TUI Writer

Help screen on how to use and customize TUI Writer.

## How to use

### Live transcriber with AI-powered editing:

- Press **SPACE** on your keyboard to start recording (top bar turns red)
- Simply start speaking and the transcriber will capture your words
- The AI automatically detects when you want to edit previous text
- Just say things like:
  - "Actually, change that to..."
  - "Delete the last sentence"
  - "Replace pizza with hamburgers"
- The AI will automatically apply edits and update the transcript
- Press **SPACE** again to stop recording

### Examples of voice commands:

**Adding new text:** Just speak normally, the AI will append it.

**Editing:** 
- "Change the word happy to excited"
- "Delete that last part"
- "Replace the first sentence with: Today was great"

## Settings & Configuration

### Audio Settings (Press 's'):
- Choose Whisper model (tiny to large-v3)
- Select transcription language
- Faster models = lower accuracy, slower models = better accuracy

### AI Model Settings (Press 'a'):
- Switch between OpenAI, Anthropic, and Google Gemini
- Choose specific models from each provider
- Configure API keys securely
- All providers work through the same unified interface

## Keyboard Shortcuts

| Key     | Action                    |
|---------|---------------------------|
| SPACE   | Start/Stop recording      |
| c       | Copy transcript           |
| s       | Audio settings            |
| a       | AI model settings         |
| ?       | This help screen          |
| q       | Quit application          |
"""

class HelpModal(ModalScreen):
    def compose(self) -> ComposeResult:
        with Container(id="help"):
            markdown_viewer = MarkdownViewer(HELP_MARKDOWN, show_table_of_contents=True)
            markdown_viewer.code_indent_guides = False
            yield markdown_viewer
            yield Center(Button("Close (esc)", variant="primary"))

    def on_button_pressed(self) -> None:
        self.app.pop_screen()

    def key_escape(self) -> None:
        self.app.pop_screen()


## TranscriptionTUI

Main terminal user interface application for audio transcription.

The TUI uses Textual's [theme system](https://textual.textualize.io/guide/design/) with built-in color variables like `$primary`, `$error`, `$success`, etc. 

**Available built-in themes:**
- `textual-dark` (default)
- `textual-light`
- `nord`
- `gruvbox`
- `tokyo-night`
- `solarized-light`

To change the theme, modify the `self.theme` assignment in `__init__`.


In [ ]:
#| export
# Constants for transcriber configuration
DEFAULT_VAD_THRESHOLD = 0.5
DEFAULT_MIN_SPEECH_DURATION_MS = 250
DEFAULT_MIN_SILENCE_DURATION_MS = 500
MIN_EDIT_INSTRUCTION_WORDS = 2

In [ ]:
#| export
class RecordingState(Enum):
    """High-level states the TUI can be in."""
    IDLE = auto()       # Not recording
    RECORDING = auto()  # Live transcription is running

class TranscriptionTUI(App):
    """Terminal user interface for live audio transcription and AI-powered editing.
    
    Features:
    - Live speech-to-text transcription using Whisper
    - AI-powered automatic edit detection and application
    - Configurable models and languages
    - Copy transcription to clipboard
    - AI model management (OpenAI/Anthropic/Gemini)
    """

    # Class attributes
    CSS = TEXTUAL_CSS
    AUTO_FOCUS = False
    state = reactive(RecordingState.IDLE)
    BINDINGS = [
        ("q", "quit", "Quit"),
        ("?", "help_modal", "Help"),
        ("s", "settings_modal", "Settings"),
        ("a", "ai_settings_modal", "AI Models"),
        ("space", "toggle_recording", "Start/Stop"),
        ("c", "copy_transcription", "Copy transcript")
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Transcription state
        self._transcription_task: asyncio.Task | None = None
        self._transcriber: LiveTranscriber | None = None

        # Configuration
        self.whisper_model = "base"
        self.language = "en"
        
        # AI provider configuration
        self.ai_provider = "openai"
        self.ai_model = "gpt-4o-mini"
        
        # Initialize transcript editor
        self.transcript_editor = None

    def compose(self) -> ComposeResult:
        """Construct and layout all UI widgets."""
        yield Header(show_clock=True)
        yield Log(id="transcript-display", auto_scroll=True)
        yield Footer()

    def _get_full_model_name(self) -> str:
        """Get the full model name in lisette format (provider/model)."""
        return f"{self.ai_provider}/{self.ai_model}"

    # === UI Actions (triggered by key bindings) ===
    def action_help_modal(self) -> None:
        self.push_screen(HelpModal())

    def action_settings_modal(self) -> None:
        self.push_screen(SettingsModal(self.whisper_model, self.language), self.apply_settings)

    def action_ai_settings_modal(self) -> None:
        """Open AI model settings modal."""
        self.push_screen(
            AISettingsModal(self.ai_provider, self.ai_model), 
            self.apply_ai_settings
        )

    async def action_toggle_recording(self) -> None:
        """Spacebar: start if idle, stop if currently recording."""
        if self.state is RecordingState.IDLE:
            # Initialize transcript editor if needed
            if not self.transcript_editor:
                self.transcript_editor = TranscriptEditor(self._get_full_model_name())
            
            await self._start(self.on_transcript_chunk)
            self.state = RecordingState.RECORDING
        elif self.state is RecordingState.RECORDING:
            self.title = "STOPPING..."
            await self._stop()
            self.state = RecordingState.IDLE

    def action_copy_transcription(self) -> None:
        if self.transcript_editor and self.transcript_editor.full_transcript:
            pyperclip.copy(self.transcript_editor.full_transcript)
            self.notify("Copied transcription", title="Copied to clipboard!")
        else:
            self.notify("Nothing to copy yet!", severity="warning", title="Oops!")

    # === Settings & Modal Callbacks ===
    def apply_settings(self, settings: list[str]) -> None:
        """Apply Whisper model settings."""
        model, language = settings
        self.whisper_model = model
        self.language = language
    
    def apply_ai_settings(self, result) -> None:
        """Apply AI provider and model settings."""
        if result:
            provider, model = result
            self.ai_provider = provider
            self.ai_model = model
            # Reinitialize transcript editor with new model
            self.transcript_editor = TranscriptEditor(self._get_full_model_name())
            # Update the title to show the new model
            if self.state == RecordingState.IDLE:
                self.title = f"○ STANDBY ({self.ai_provider}/{self.ai_model})"

    # === Transcription Lifecycle ===
    async def _start(self, on_transcript_callback) -> None:
        self._transcriber = LiveTranscriber(
            model_id=self.whisper_model,
            language=self.language,
            on_transcript=on_transcript_callback,
            vad_threshold=DEFAULT_VAD_THRESHOLD,
            min_speech_duration_ms=DEFAULT_MIN_SPEECH_DURATION_MS,
            min_silence_duration_ms=DEFAULT_MIN_SILENCE_DURATION_MS,
        )
        self._transcription_task = asyncio.create_task(self._transcriber.start())

    async def _stop(self) -> None:
        try:
            if self._transcriber:
                await self._transcriber.stop()
            if self._transcription_task:
                await self._transcription_task
        finally:
            self._transcriber = None
            self._transcription_task = None

    # === Event Handlers & Callbacks ===
    def _normalize_text(self, text: str) -> str:
        """Normalize and clean text input."""
        return (text or "").strip()

    def on_transcript_chunk(self, text: str) -> None:
        """Called whenever the transcriber produces a new text chunk.
        Passes through AI to detect edits vs new text."""
        text = self._normalize_text(text)
        if not text:
            return
        
        try:
            # Process chunk through AI
            result = self.transcript_editor.process_chunk(text)
            
            # Update display based on action
            if result["action"] == "edit":
                # Full transcript was edited - refresh entire display
                self.transcript_display.clear()
                self.transcript_display.write_lines(result["transcript"].splitlines(True))
                self.notify(f"✏️ Edit detected (tokens: {result['tokens_used']})", timeout=2)
            else:
                # New text appended - just add the line
                self.transcript_display.write_line(text)
            
        except Exception as e:
            # Fallback: just append the text without AI processing
            self.notify(f"AI processing failed: {str(e)}", severity="error", timeout=3)
            self.transcript_display.write_line(text)

    def watch_state(self, new_state: RecordingState) -> None:
        """Reactively update UI when recording state changes."""
        self.sub_title = ""

        match new_state:
            case RecordingState.IDLE:
                self.title = f"○ STANDBY ({self.ai_provider}/{self.ai_model})"
                self.header.remove_class("recording")
            case RecordingState.RECORDING:
                self.title = "● RECORDING"
                self.header.add_class("recording")

    # === Lifecycle Hooks ===
    def on_mount(self) -> None:
        """Initialize widget references and set titles."""
        self.title = f"○ STANDBY ({self.ai_provider}/{self.ai_model})"
        self.header = self.query_one(Header)
        self.transcript_display: Log = self.query_one("#transcript-display", Log)
        self.theme = "textual-dark"

    async def on_unmount(self) -> None:
        await self._stop()
